In [92]:
import requests
import pandas as pd

league_id = 815838
BASE_URL = "https://fantasy.premierleague.com/api"

bootstrap = requests.get(f"{BASE_URL}/bootstrap-static/").json()

players = {p["id"]: p for p in bootstrap["elements"]}
teams_lookup = {t["id"]: t["name"] for t in bootstrap["teams"]}
current_gw = next(event["id"] for event in bootstrap["events"] if event["is_current"])

In [93]:
def get_formation(picks):
    formation = {"DEF": 0, "MID": 0, "FWD": 0}

    for pick in picks["picks"]:
        if pick["position"] <= 11:  # starters only - to counter bboost
            if pick["element_type"] == 2:  # DEF
                formation["DEF"] += 1
            elif pick["element_type"] == 3:  # MID
                formation["MID"] += 1
            elif pick["element_type"] == 4:  # FWD
                formation["FWD"] += 1

    return f"{formation['DEF']}-{formation['MID']}-{formation['FWD']}"

def get_full_league_standings_and_name(league_id: int):
    standings = []
    page = 1
    while True:
        url = f"{BASE_URL}/leagues-classic/{league_id}/standings/?page_standings={page}"
        resp = requests.get(url).json()
        
        results = resp["standings"]["results"]
        standings.extend(results)
        
        if resp["standings"]["has_next"]:
            page += 1
        else:
            break

    league_name = resp["league"]["name"]
    
    return standings, league_name

Form the teams table

In [94]:
# Fetch league standings
league, league_name = get_full_league_standings_and_name(league_id)
entries = [team["entry"] for team in league]
gw = current_gw

teams_data = []
all_picks = []

for entry_id in entries:
    entry_info = requests.get(f"{BASE_URL}/entry/{entry_id}/").json()
    player_name = f"{entry_info['player_first_name']} {entry_info['player_last_name']}"
    favourite_team = teams_lookup[entry_info.get("favourite_team")] if entry_info.get("favourite_team") else None
    picks = requests.get(f"{BASE_URL}/entry/{entry_id}/event/{gw}/picks/").json()
    all_picks.append(picks)
    transfers = requests.get(f"{BASE_URL}/entry/{entry_id}/transfers/").json()
    
    # Transfers of recent GW
    gw_transfers = [
        (players[t["element_in"]]["web_name"], 
         players[t["element_out"]]["web_name"], 
         players[t["element_in"]]["total_points"] - players[t["element_out"]]["total_points"])
        for t in transfers if t["event"] == gw
    ]
    
    # Formation
    formation = get_formation(picks)
    
    # Captain & VC
    captain = players[next(p["element"] for p in picks["picks"] if p["is_captain"])]["web_name"]
    vice_captain = players[next(p["element"] for p in picks["picks"] if p["is_vice_captain"])]["web_name"]

    playing_XI = [players[p["element"]]["web_name"] for p in picks['picks'] if p["multiplier"] != 0]
    bench = [players[p["element"]]["web_name"] for p in picks['picks'] if p["multiplier"] == 0]

    teams_data.append({
        "Entry ID": entry_id,
        "Player name": player_name,
        "Team name": entry_info["name"],
        "Favourite team": favourite_team,
        "Total points": entry_info["summary_overall_points"],
        "GW points": entry_info["summary_event_points"],
        "Transfers": gw_transfers,
        "Formation": formation,
        "Captain": captain,
        "Vice Captain": vice_captain,
        "Playing XI": playing_XI,
        "Bench": bench,
        "Chips used": picks.get("active_chip")
    })

df_teams = pd.DataFrame(teams_data)

Add ranking history data to the dataframe

In [95]:
# dict to hold rank history per entry
rank_history = {}

# loop over GWs
for gw in range(1, current_gw + 1):
    url = f"{BASE_URL}/leagues-classic/{league_id}/standings/?event={gw}"
    resp = requests.get(url).json()

    for result in resp.get("standings", {}).get("results", []):
        entry_id = result["entry"]
        rank = result["rank"]

        # init list if not present
        if entry_id not in rank_history:
            rank_history[entry_id] = []
        
        rank_history[entry_id].append(rank)

df_rank_history = pd.DataFrame(list(rank_history.items()), columns=["Entry ID", "Rankings history"])

# Merge with your main df
df_teams = df_teams.merge(df_rank_history, on="Entry ID", how="left")

Add adjacent teams points difference

In [96]:
diff_col = df_teams["Total points"].diff().fillna(0).astype(int)

# Insert diff column *after* "value"
col_position = df_teams.columns.get_loc("Total points") + 1
df_teams.insert(col_position, "Adjacent points difference", diff_col)

In [97]:
df_teams

,Entry ID,Player name,Team name,Favourite team,Total points,Adjacent points difference,GW points,Transfers,Formation,Captain,Vice Captain,Playing XI,Bench,Chips used,Rankings history
0,3920569,Eric Zurita,Hustle & Flo,Liverpool,78,0,78,[],3-4-3,Haaland,Wirtz,"[Raya, Andersen, Aït-Nouri, Muñoz, Wirtz, Palm...","[Areola, Baleba, Alex Moreno, Tuanzebe]",None,[1]
1,4013674,Kait Wojtaszek,K8 the Gr8,Arsenal,77,-1,77,[],4-4-2,M.Salah,Wood,"[Pickford, Schär, Gabriel, Tarkowski, Aina, Cu...","[Verbruggen, Ndiaye, Alex Moreno, Kostoulas]",None,[2]
2,4140344,Gerd Woort-Menker,IwobiKlose,None,60,-17,60,[],3-5-2,M.Salah,Wirtz,"[Verbruggen, Frimpong, Murillo, Tarkowski, Wir...","[Dúbravka, Piroe, Gudmundsson, Reinildo]",None,[3]
3,5304099,D'Arcy Williams,TheatreOfMemes,Man Utd,55,-5,55,[],4-4-2,M.Salah,Palmer,"[Sánchez, Van de Ven, Wan-Bissaka, Konsa, Frim...","[Dúbravka, Estève, A.Ramsey, Marc Guiu]",None,[4]
4,7888775,Kamal Logue,Saka Potatoes,Chelsea,53,-2,53,[],4-4-2,Delap,Pickford,"[Pickford, James, Van de Ven, Wan-Bissaka, Rom...",[],bboost,[5]
5,4540331,Tim Woodhouse,All Change,Spurs,51,-2,51,[],3-4-3,Palmer,João Pedro,"[Sánchez, J.Timber, Frimpong, Pedro Porro, Pal...","[Dúbravka, Konsa, Malen, Wan-Bissaka]",None,[6]
6,3733935,Stefan Menker,EZe-pass,Arsenal,49,-2,49,[],4-4-2,Watkins,Palmer,"[Dúbravka, Murillo, Tarkowski, Virgil, Pedro P...","[Areola, Estève, Rice, Barnes]",None,[7]
7,4993729,Rick Thorley,Gameofthrowins,None,49,0,49,[],4-4-2,M.Salah,Palmer,"[Sánchez, Virgil, Wan-Bissaka, Murillo, Tarkow...","[Dúbravka, Hill, Marc Guiu, G.Rodriguez]",None,[7]
8,7547590,John Deutsch,Deutschmeister,Arsenal,48,-1,48,[],3-5-2,M.Salah,Palmer,"[Sánchez, Murillo, Pedro Porro, Andersen, M.Sa...","[Dúbravka, Dorgu, Estève, Marc Guiu]",None,[9]
9,9114697,Christopher Kelly Jr,Elland Roadie,Man Utd,44,-4,44,[],4-4-2,Bowen,Aït-Nouri,"[Sels, Cucurella, Kerkez, Pedro Porro, Aït-Nou...","[Sánchez, J.Timber, Cunha, Raúl]",None,[10]


Form the footballers table

In [98]:
from collections import defaultdict

picks_count = defaultdict(int)
captain_count = defaultdict(int)

for team in all_picks:
    active_picks = {p["element"]: p for p in team["picks"]}

    # Determine who was captain + vice
    cap_id = next(p["element"] for p in team["picks"] if p["is_captain"])
    vice_id = next(p["element"] for p in team["picks"] if p["is_vice_captain"])

    # Fetch autosubs info (to check if captain missed out)
    autosubs = team.get("automatic_subs", [])

    # Count all picks
    for p in active_picks.values():
        picks_count[p["element"]] += 1

    # If captain is autosubbed out, vice takes over
    cap_out = any(s["element_out"] == cap_id for s in autosubs)
    if cap_out:
        vice_out = any(s["element_out"] == vice_id for s in autosubs)
        if vice_out: #Case where both C and VC don't play
            None
        else:
            captain_count[vice_id] += 1
    else:
        captain_count[cap_id] += 1

# ===== 5. Build DataFrame =====
footballers_data = []
for p in bootstrap["elements"]:
    footballers_data.append({
        "Footballer ID":p["id"] ,
        "Footballer name": p["web_name"],
        "Total points": p["total_points"],
        "GW points": p["event_points"],
        "Real team name": teams_lookup[p["team"]],
        "Real team ID": p["team"],
        "Price (in Millions £)": p["now_cost"] / 10,
        "Price last GW (in Millions £)": p["cost_change_event"] / 10 + (p["now_cost"] / 10),
        "Price difference (in Millions £)": p["cost_change_event"] / 10,
        "Times chosen in squad": picks_count[p["id"]],
        "Times captained": captain_count[p["id"]]
    })

df_footballers = pd.DataFrame(footballers_data)

In [99]:
df_footballers.iloc[420:470]

,Footballer ID,Footballer name,Total points,GW points,Real team name,Real team ID,Price (in Millions £),Price last GW (in Millions £),Price difference (in Millions £),Times chosen in squad,Times captained
420,402,Aït-Nouri,9,9,Man City,13,6.0,6.0,0.0,2,0
421,403,Gvardiol,0,0,Man City,13,5.9,5.8,-0.1,0,0
422,404,Akanji,0,0,Man City,13,5.5,5.5,0.0,0,0
423,405,Aké,0,0,Man City,13,5.5,5.5,0.0,0,0
424,406,Khusanov,1,1,Man City,13,5.5,5.5,0.0,0,0
425,407,Matheus N.,1,1,Man City,13,5.5,5.5,0.0,0,0
426,408,Rúben,6,6,Man City,13,5.5,5.5,0.0,0,0
427,409,Stones,6,6,Man City,13,5.5,5.5,0.0,0,0
428,410,Lewis,11,11,Man City,13,5.0,5.0,0.0,0,0
429,411,O’Reilly,0,0,Man City,13,5.0,5.0,0.0,0,0


------------ Metrics -----------

a) Rules based

Helper functions

In [100]:
def all_extremes(series, metric="max", use_abs=False):
    values = series.abs() if use_abs else series
    
    if metric == "max":
        extreme_value = values.max()
    elif metric == "min":
        extreme_value = values.min()
    else:
        raise ValueError("metric must be 'max' or 'min'")
    
    return series[values == extreme_value].index.tolist()

def names_from_indices(df, indices, column="Team name"):
    return [df.iloc[i][column] for i in indices if i < len(df)]

def value_from_first_index(df, indices, column):
    if indices:
        return df.iloc[indices[0]][column]
    return None

Logic

In [101]:
rule_based_metrics = []

# 1 League topper
idx_league_topper = all_extremes(df_teams["Total points"], "max")
all_league_toppers = names_from_indices(df_teams, idx_league_topper, "Team name")
rule_based_metrics.append(f"{all_league_toppers} topped the league")

# 2 League bottom
idx_league_bottom = all_extremes(df_teams["Total points"], "min")
all_league_bottoms = names_from_indices(df_teams, idx_league_bottom, "Team name")
rule_based_metrics.append(f"{all_league_bottoms} are at the bottom of the league")

# 4 Max change in GW points (absolute)
idx_max_change = all_extremes(df_teams["GW points"], "max", use_abs=True)
all_teams_max_change = names_from_indices(df_teams, idx_max_change, "Team name")
value_max_change = value_from_first_index(df_teams, idx_max_change, "GW points")

if value_max_change is not None:
    for idx in idx_max_change:
        total_points = int(df_teams.iloc[idx]["Total points"])
        prev_points = total_points - int(df_teams.iloc[idx]["GW points"])
        max_change_in_points = (
            f"{all_teams_max_change} showed the maximum change of points over the last week. "
            f"Their change: {value_max_change} points from {prev_points} to {total_points}."
        )
        rule_based_metrics.append(max_change_in_points)

# 5 Most points gained
idx_most_points_gained = all_extremes(df_teams["GW points"], "max")
all_teams_most_points_gained = names_from_indices(df_teams, idx_most_points_gained, "Team name")
value_most_points_gained = value_from_first_index(df_teams, idx_most_points_gained, "GW points")
if value_most_points_gained is not None:
    rule_based_metrics.append(
        f"{all_teams_most_points_gained} gained the most points ({value_most_points_gained}) this Gameweek."
    )

# 6 Least points gained
idx_least_points_gained = all_extremes(df_teams["GW points"], "min")
all_teams_least_points_gained = names_from_indices(df_teams, idx_least_points_gained, "Team name")
value_least_points_gained = value_from_first_index(df_teams, idx_least_points_gained, "GW points")
if value_least_points_gained is not None:
    rule_based_metrics.append(
        f"{all_teams_least_points_gained} gained the least points ({value_least_points_gained}) this Gameweek."
    )

# 7 Highest scoring footballer
idx_highest_scoring_footballer = all_extremes(df_footballers["GW points"], "max")
all_highest_scoring_footballers = names_from_indices(df_footballers, idx_highest_scoring_footballer, "Footballer name")
value_highest_scoring_footballers = value_from_first_index(df_footballers, idx_highest_scoring_footballer, "GW points")
if value_highest_scoring_footballers is not None:
    rule_based_metrics.append(
        f"{all_highest_scoring_footballers} gained the most points ({value_highest_scoring_footballers}) this Gameweek."
    )

# 8 Lowest scoring footballer
idx_lowest_scoring_footballer = all_extremes(df_footballers["GW points"], "min")
all_lowest_scoring_footballers = names_from_indices(df_footballers, idx_lowest_scoring_footballer, "Footballer name")
value_lowest_scoring_footballers = value_from_first_index(df_footballers, idx_lowest_scoring_footballer, "GW points")
if value_lowest_scoring_footballers is not None:
    rule_based_metrics.append(
        f"{all_lowest_scoring_footballers} gained the least points ({value_lowest_scoring_footballers}) this Gameweek."
    )

# 9 Highest scoring real team
team_scores = {
    team_id: df_footballers.loc[df_footballers["Real team ID"] == team_id, "GW points"].sum()
    for team_id in range(1, 21)
}
if team_scores:
    max_score = max(team_scores.values())
    max_teams = [tid for tid, score in team_scores.items() if score == max_score]
    highest_scoring_real_teams_list = [teams_lookup[tid] for tid in max_teams]
    rule_based_metrics.append(f"{highest_scoring_real_teams_list} scored the most points ({max_score}).")

# 10 Lowest scoring real team
if team_scores:
    min_score = min(team_scores.values())
    min_teams = [tid for tid, score in team_scores.items() if score == min_score]
    lowest_scoring_real_teams_list = [teams_lookup[tid] for tid in min_teams]
    rule_based_metrics.append(f"{lowest_scoring_real_teams_list} scored the least points ({min_score}).")

# 11-12 Best/worst transfers
max_value, min_value = float("-inf"), float("inf")
max_transfers, min_transfers = [], []

for idx, row in df_teams.iterrows():
    for tup in row.get("Transfers", []):
        x = tup[2]
        if x > max_value:
            max_value, max_transfers = x, [(idx, tup)]
        elif x == max_value:
            max_transfers.append((idx, tup))
        if x < min_value:
            min_value, min_transfers = x, [(idx, tup)]
        elif x == min_value:
            min_transfers.append((idx, tup))

for idx, tup in max_transfers:
    team_name = df_teams.iloc[idx]["Team name"]
    rule_based_metrics.append(
        f"{team_name} got {tup[0]} in and removed {tup[1]} smartly and saw a change of {max_value} points."
    )

for idx, tup in min_transfers:
    team_name = df_teams.iloc[idx]["Team name"]
    rule_based_metrics.append(
        f"{team_name} got {tup[0]} in and removed {tup[1]} unwisely and saw a change of {min_value} points."
    )

# 13 Highest price increase
idx_highest_increased_price = all_extremes(df_footballers["Price difference (in Millions £)"], "max")
all_highest_price_increase_footballers = [
    df_footballers.iloc[i]["Footballer name"]
    for i in idx_highest_increased_price
    if df_footballers.iloc[i]["Price difference (in Millions £)"] > 0
]
value_highest_increased_price = value_from_first_index(df_footballers, idx_highest_increased_price, "Price difference (in Millions £)")
if all_highest_price_increase_footballers and value_highest_increased_price is not None:
    rule_based_metrics.append(
        f"{all_highest_price_increase_footballers} had the highest price increase (£{value_highest_increased_price}M) this Gameweek."
    )

# 14 Highest priced players vs points
most_expensive_players = df_footballers.sort_values("Price (in Millions £)", ascending=False).head(5)
price_vs_points = "Highest priced footballers \n"
for _, row in most_expensive_players.iterrows():
    if players.get(row["Footballer ID"], {}).get("minutes", 0) == 0:
        continue
    price_vs_points += f"{row['Footballer name']} valued at {row['Price (in Millions £)']} scored {row['GW points']} points. \n"
rule_based_metrics.append(price_vs_points.strip())

# 15 Most chosen players vs points
most_chosen_players = df_footballers.sort_values("Times chosen in squad", ascending=False).head(5)
chosen_vs_points = "Most picked footballers \n"
for _, row in most_chosen_players.iterrows():
    if players.get(row["Footballer ID"], {}).get("minutes", 0) == 0:
        continue
    chosen_vs_points += f"{row['Footballer name']} chosen {row['Times chosen in squad']} times scored {row['GW points']} points. \n"
rule_based_metrics.append(chosen_vs_points.strip())

# 16 Most captained players vs points
most_captained_players = df_footballers.sort_values("Times captained", ascending=False).head(5)
captained_vs_points = "Most times picked as captains \n"
for _, row in most_captained_players.iterrows():
    if players.get(row["Footballer ID"], {}).get("minutes", 0) == 0:
        continue
    captained_vs_points += f"{row['Footballer name']} chosen captain {row['Times captained']} times scored {row['GW points']} points. \n"
rule_based_metrics.append(captained_vs_points.strip())

# 18 Chips usage
chips_usage = []
for _, row in df_teams.iterrows():
    if row.get("Chips used"):
        chips_usage.append(f"{row['Team name']} used the chip(s): {row['Chips used']}")
if chips_usage:
    rule_based_metrics.append(" ".join(chips_usage))

# 19 Rare players (least selected in starting XI across league)
all_playing_times_played = {}

# Collect all players that actually played (multiplier > 0)
for pick in all_picks:
    for player in pick.get("picks", []):
        if player.get("multiplier", 0) != 0:
            row = df_footballers.loc[df_footballers["Footballer ID"] == player["element"], "Times chosen in squad"]
            if not row.empty:
                all_playing_times_played[player["element"]] = int(row.squeeze())

if all_playing_times_played:
    min_selected = min(all_playing_times_played.values())
    least_selected_players = {
        pid: count for pid, count in all_playing_times_played.items() if count == min_selected
    }

    # Build {player name: score}
    least_selected_players_vs_scores = {}
    for pid in least_selected_players:
        name = players[pid]["web_name"]
        score = players[pid].get("event_points", 0)
        least_selected_players_vs_scores[name] = score

    rare_players_score = (
        f"Here is/are the least selected player(s) who started {min_selected} times "
        f"and their score(s): {least_selected_players_vs_scores}."
    )
    rule_based_metrics.append(rare_players_score)
else:
    rule_based_metrics.append("No rare players found this Gameweek.")

In [102]:
rule_based_metrics

["['Hustle & Flo'] topped the league",
 "['People on the pitch', 'Ha-Cunha Mateta'] are at the bottom of the league",
 "['Hustle & Flo'] showed the maximum change of points over the last week. Their change: 78 points from 0 to 78.",
 "['Hustle & Flo'] gained the most points (78) this Gameweek.",
 "['People on the pitch', 'Ha-Cunha Mateta'] gained the least points (40) this Gameweek.",
 "['Ballard'] gained the most points (17) this Gameweek.",
 "['Doherty'] gained the least points (-1) this Gameweek.",
 "['Man City'] scored the most points (86).",
 "['West Ham', 'Wolves'] scored the least points (18).",
 "['Semenyo', 'Ekitiké', 'Reijnders', 'Wood', 'Ballard'] had the highest price increase (£0.1M) this Gameweek.",
 'Highest priced footballers \nM.Salah valued at 14.5 scored 8 points. \nHaaland valued at 14.0 scored 13 points. \nPalmer valued at 10.5 scored 3 points. \nSaka valued at 10.0 scored 3 points.',
 'Most picked footballers \nPalmer chosen 11 times scored 3 points. \nDúbravka ch

b) LLM based

In [103]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [104]:
rule_based_metrics_text = "\n".join(f"- {metric}" for metric in rule_based_metrics)
df_teams_text = df_teams.to_string(index=False)

In [105]:
prompt = f"""You have to write a commissioner-style roundup commentary for our Fantasy Premier League private league Gameweek 1 in 500-700 words. I am 
providing you some highlight-metrics that I could find from my data. I am also giving you the standings table of my league which 
you need to observe and make comments about. You also need to have knowledge of the Premier Leagure footballing world and make 
remarks that the fans can relate to, as a part of your sense of humor. 

Here are some of the highlights: {rule_based_metrics_text}
Mention most of these stats in the summary, depending on how interesting each is. Remember to mention table topper, last position holder, highest scoring footballer every time.
For example, looking at the player prices vs their points, you can comment on their performance and what was expected from them. Similarly for most chosen players, and most captained players.
Another thing you could comment on is how the least selected players performed, and make sure to mention which player(s) chose them in their team and benefitted from it. 
You can see team line-ups in the dataframe given later. 

Here is a teams table. You must use it to comment on the structure of the standings table, and how competitive what part of the table is: {df_teams_text}
Using the table, also look for unusual stuff, or blunders that the players might have made for example not setting up a captain 
or vice-captain, playing injured/suspended players or the ones in transfer talks and not available for selection.
In the table you would see player names as well, you can use them interchangeably with the team names for a more personal effect, but only sometimes.
You also have a favourite teams column for every player, feel free to trash talk a player if their favourite team got bashed in the GW.
You also have the rankings history of every team in a list. Eg. [2, 4, 3] means a team ranked 2nd, 4th, and 3rd, after the first, second and third GW respectively.

Use the metrics and the table provided to generate a commissioner styled FPL roundup for this GameWeek. Feel free to be creative!
"""

In [63]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a witty fantasy football commissioner."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
)

print(response.choices[0].message.content)

Welcome, ladies and gentlemen, to the opening salvo of our Fantasy Premier League private league! Gameweek 1 has been a rollercoaster ride akin to watching a Tottenham Hotspur title challenge: full of promise at the start, but we all know how these things usually end. Let’s dive into the drama, the triumphs, and the ‘what-were-they-thinking’ moments that have left us both chuckling and questioning our life choices.

**Top of the Pops: Hustle & Flo**

Kicking things off with a bang, Eric Zurita and his squad, ‘Hustle & Flo,’ danced their way to the top, scoring a sensational 78 points. It’s almost as if they borrowed some of Liverpool’s rhythm and applied it here, leaving the rest of us to wonder if they’ve got a secret playlist we’re missing out on. With Haaland as captain netting 13 points and a lineup that included the ever-reliable Wirtz and João Pedro, Eric has set the bar high. Note to self: whatever Eric’s drinking, I’ll have two!

**Scraping the Bottom: People on the Pitch & Ha-

Extra statistics to display on the roundup

1. Top 3, Bottom 3

In [64]:
top_3_df = df_teams.sort_values("Total points", ascending=False).head(3)[["Player name", "Team name", "Total points"]]
bottom_3_df = df_teams.sort_values("Total points", ascending=False).tail(3)[["Player name", "Team name", "Total points"]]

In [65]:
top_3_df

,Player name,Team name,Total points
0,Eric Zurita,Hustle & Flo,78
1,Kait Wojtaszek,K8 the Gr8,77
2,Gerd Woort-Menker,IwobiKlose,60


In [66]:
bottom_3_df

,Player name,Team name,Total points
10,Jasmine Benzine,Arroz Blanco FC,43
11,Paul Quinsee,People on the pitch,40
12,Mark Woort-Menker,Ha-Cunha Mateta,40


2. Best/Worst transfer making teams

In [67]:
# Best and worst transfer-making teams (net score)
team_transfer_scores = {}

for idx, row in df_teams.iterrows():
    transfers = row.get("Transfers", [])
    total_score = sum(tup[2] for tup in transfers) if transfers else 0
    team_transfer_scores[idx] = total_score

best_transfer_str, worst_transfer_str = "", ""

if team_transfer_scores:  # safety check
    max_score = max(team_transfer_scores.values())
    min_score = min(team_transfer_scores.values())

    best_teams = [df_teams.iloc[idx]["Team name"] for idx, score in team_transfer_scores.items() if score == max_score]
    worst_teams = [df_teams.iloc[idx]["Team name"] for idx, score in team_transfer_scores.items() if score == min_score]

    # Create the required strings
    best_transfer_str = f"{', '.join(best_teams)} : {max_score}"
    worst_transfer_str = f"{', '.join(worst_teams)} : {min_score}"

3. League Name, Gameweek Number

In [73]:
league

[{'id': 32189909,
  'event_total': 78,
  'player_name': 'Eric Zurita',
  'rank': 1,
  'last_rank': 0,
  'rank_sort': 1,
  'total': 78,
  'entry': 3920569,
  'entry_name': 'Hustle & Flo',
  'has_played': True},
 {'id': 39205524,
  'event_total': 77,
  'player_name': 'Kait Wojtaszek',
  'rank': 2,
  'last_rank': 0,
  'rank_sort': 2,
  'total': 77,
  'entry': 4013674,
  'entry_name': 'K8 the Gr8',
  'has_played': True},
 {'id': 31292004,
  'event_total': 60,
  'player_name': 'Gerd Woort-Menker',
  'rank': 3,
  'last_rank': 0,
  'rank_sort': 3,
  'total': 60,
  'entry': 4140344,
  'entry_name': 'IwobiKlose',
  'has_played': True},
 {'id': 41116754,
  'event_total': 55,
  'player_name': "D'Arcy Williams",
  'rank': 4,
  'last_rank': 0,
  'rank_sort': 4,
  'total': 55,
  'entry': 5304099,
  'entry_name': 'TheatreOfMemes',
  'has_played': True},
 {'id': 64885607,
  'event_total': 53,
  'player_name': 'Kamal Logue',
  'rank': 5,
  'last_rank': 0,
  'rank_sort': 5,
  'total': 53,
  'entry': 788

In [69]:
final_response = f"\t\t\t Summary: \n\n{response.choices[0].message.content} \n\nTop 3 teams: \n{top_3_df} \n\nBottom 3 teams: \n{bottom_3_df} \n\nBest Transfer Maker(s): \n{best_transfer_str} \n\nWorst Transfer Maker(s): \n{worst_transfer_str}"

In [70]:
print(final_response)

			 Summary: 

Welcome, ladies and gentlemen, to the opening salvo of our Fantasy Premier League private league! Gameweek 1 has been a rollercoaster ride akin to watching a Tottenham Hotspur title challenge: full of promise at the start, but we all know how these things usually end. Let’s dive into the drama, the triumphs, and the ‘what-were-they-thinking’ moments that have left us both chuckling and questioning our life choices.

**Top of the Pops: Hustle & Flo**

Kicking things off with a bang, Eric Zurita and his squad, ‘Hustle & Flo,’ danced their way to the top, scoring a sensational 78 points. It’s almost as if they borrowed some of Liverpool’s rhythm and applied it here, leaving the rest of us to wonder if they’ve got a secret playlist we’re missing out on. With Haaland as captain netting 13 points and a lineup that included the ever-reliable Wirtz and João Pedro, Eric has set the bar high. Note to self: whatever Eric’s drinking, I’ll have two!

**Scraping the Bottom: People on 